<a href="https://colab.research.google.com/github/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/Quantum_circuit_optimization_with_Deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Circuit optimization with Deep learning

This is a educational notebook about what is quantum circuit optimization and how to optimize the circuit using deep learning approach created for IBM Q Challenge teach me qiskit award

0. Welcome :D
1. What is circuit optimization?
2. How does optimized circuit looks like depends on hardware?
3. Check quantum hardware information
4. How to generate random circuit?
5. Generate dataset for Deep learning
6. Train LSTM Autoencoder to find circuit manifold


    6-1. (Optional) Estimating Statevector from circuit


7. Train Layout predicting Model
8. Train Optimized circuit generating model
9. How to check it is optimized?



A. Future work

## 0. Welcome :D

This topic include big main things!

1. Deep learning 
2. Qiskit


    If you never heard about those no problem!
    This notebook includes several sub notebooks and you can simply
    check one by one executing the cell without prior knowledge
    But it will be helpful if you know what is what.
    Recommend reading following chapter at qiskit textbook before you start
    
    chapter 0. Prerequisites
    chapter 1. Quantum States and Qubits 
    chapter 2. Single Qubits and Multiple-Qubits gates

[QISKIT TESTBOOK](https://community.qiskit.org/textbook/preface.html)

## 1. What is circuit optimization?

    Circuit is a program in quantum computer
    Circuit optimization is process making the program fast and
    accurate by making the total noise inside small as possible
    You will learn how the program is optimized

Most of the explaination includes classic computer example

    Also don't forget to look at IBM transpiler jupyter notebook
    linked right after this cell.
    It is very useful to understand circuit optimization





This is Good introduction from IBM about circuit optimization. **Don't forget to read this!**

    https://github.com/Qiskit/qiskit-iqx-tutorials/blob/master/qiskit/fundamentals/5_using_the_transpiler.ipynb


### Circuit optimization is to reduce the total noise of circuit

#### What is noise?



    It is how much you loss your energy you stored inside the qubit.
    Energy is same as information and qubit loss energy very easy so it
    loss information very easy. You can loss energy in any way.
    Applying gate and reading the data from the qubit, also the
    energy naturally disappear.

Let's think in a classic computer term

    # Memory address is 0xA (noise 50%)
    float number_A = 1
    number_A += 1

    # Memory address is 0xB (noise 5%)
    float number_B = 1
    number_B += 1

    # Print out the result
    print(number_A, number_B)

Then you must expect both result is **(2.0, 2.0)**

    # number_A = (1 + 1) * 0.5
    # number_B = (1 + 1) * 0.95 
    But the result is (1.0, 1.9)

If you do one more time

    # number_A = (1 + 1) * 0.5
    # number_B = (1.9 + 1) * 0.95 
    The result will be (1.0, 2.755)

1. **Choosing which memory address** to use is choosing which qubit to use
2. **+= operation** is gate operator applied to the qubit




#### What is circuit optimization?



`Below phrase is what IBM mentioned`

Optimize circuits to minimize noise when they are executed on a real IBMQ backend, by using the properties of the backends themselves. These properties contain information about the physics of the device, such as qubit lifetimes (decoherence and relaxation), gate and readout error rates, and gate latencies.

    1. Reducing gate numbers and depth

Let's think in classic term.

    # Memory address is 0xA (noise 50%)
    float number_A = 1

    # number_3_op and number_1_op is same
    # But number_3_operation applied unneccesary -1+1 operation
    number_3_op = number_A + 1 - 1 + 1
    number_1_op = number + 1

Because of each each operation has noise even it should be same the final result of two will be different.

    The result will be...
    # number_3_op
    1st op | 1 <= (1 + 1) * 0.5
    2nd op | 0 <= (1 - 1) * 0.5
    3rd op | 0.5 <= (0 + 1) * 0.5
    number_3_op -> 0.5

    # number_1_op
    1st op | 1 <= (1 + 1) * 0.5
    number_1_op -> 1

In other words minimizing noise means making circuits total gate number and depth small as possible. Applying gate to qubit cause noise so if the number of applying gates is small then noise will going to be more smaller.

    2. Applying gate to the qubit that has least noise

Let's think in classic term

    This case is already explained in upper example of number A & B
    memory 0xA - 50% noise
    memory 0xB - 5% noise

    Applying operation to memory 0xB will be surely good

Circuit optimization doesn't only focus on reducing the number of gates. It also focus on the where to put the gate at which qubit. Let's assume we have U1 gate and there is 2 qubits. qubit A will going to have 50% noise on U1 and qubit B has 5% noise on U1. Then we should put the U1 gate at qubit B to make less noise.

**Final goal of circuit optimization is to make the total noise small as possible.**


### Circuit optimization at QISKIT

#### QISKIT transpiler optimization process


Each step is well explained in the jupyter notebook in the upper link

    1. Unroll to basis gates
    2. Set initial layout
    3. Topology mapping
    4. Gate optimization

<img src="https://user-images.githubusercontent.com/7631333/66371523-f5a44480-e9d5-11e9-8a79-742dcd35d050.png" width=800px>














#### 1. Unroll to basis gate


Let's think in classic term

let's assume you have program

    def add(x1, x2):
        return (x1 + x2)
    A = add(1, 2)
    B = add(3, 4)
    C = add(A, B)

Computer cannot understand what is **function add**. It can only understand **+** So we have to change the code to only using **+** 

    # Unrolling is ongoing...
    A = 1 + 2
    B = 3 + 4
    C = A + B

#### 2. Set initial layout



You have program algorithm. But what memory space of computer you want to use? Layout choose which physical memory space to use.

Let's assume you have 3 bits memory.

    # 0xA - 50% noise
    # 0xB - 5% noise
    # 0xC - 0% noise

You have to put A, B, C. where you want to put in?

    0xA (50% noise) - A = (1 + 2) 1 operation
    0xB (5% noise) - B = (3 + 4) 1 operation
    0xC (0% noise) - C = (A + B) 3 operation

C has the most operations and most information so seems fair to put in the least 0xC which has least noise. 

    # The reason B is at 0xB is because B is holding more value than A
    # A = 3, B = 7
    # In upper case A noise is 50% so 1.5 = (1 + 2) * 0.5 
    # B will be 6.65 = (3 + 4) * 0.95
    # TOTAL NOISE LOSS = 1.5 + 0.35 = 1.85

    # When we do opposite
    # 0xA - B 0xB - A
    # A will be 2.85 = (1 + 2) * 0.95
    # B will be 3.5 = (3 + 4) * 0.5
    # TOTAL NOISE LOSS = 0.15 + 3.5 = 3.65

Even it has same number of operation the total noise could different by where do I put which variable to which memory.

    Variable could be considered logical qubit
    Memory could be considered physical qubit

**layout is choosing which logical qubit to put at which physical qubit**

    {logical qubit: physical qubit} = {Variable: Memory}


#### 3. Topology mapping



After you choose which memory to use you will find out that you cannot directly execute the program. Because the the operations written in the program could only be used in some specific memories. So you should fix your program to make your program fit the hardware constraint.

**Fixing the program to meet the hardware constraint is called topology mapping**

    Imagine you loaded two value to memory but if you want to apply
    ADD operation then you should move your value to the register
    inside CPU. It's time to consider which is the best path is the 
    best way to send the values to the CPU register

Refer **Different Variants of the Same Pass** in the following link
You can check out the swap gates is applied to meet the cx constraint between two qubits. Check out coupling map also.

    https://github.com/Qiskit/qiskit-iqx-tutorials/blob/master/qiskit/advanced/terra/4_transpiler_passes_and_passmanager.ipynb



This part is good to just know by quantum computer example related to coupling map and cx constraint. Now you have your program and mapped which variable(logical qubit) to which memory(physical qubit).

**You want use CX [3,1]**

    There are limitation of applying CX gate in real quantum computer
    if you want to apply cnot gate from qubit 3 to qubit 1 then the
    quantum computer should support it. If not then you should swap
    the qubits to meet the quantum computer CX gate constraint.

    If quantum computer only supports CX [2, 1]
    You should swap qubit 3 and qubit 2 and apply CX to it.

Swap gate includes 3 CX gates so now we have to choose the use the least swap gate as possible to swap qubits while meeting the constraint of CX gate used. This is Topology mapping. As you can see in the picture, CX gate is only can be used for two qubits that is connected with line.

![IBM's 10 Quantum Device Lineup](https://www.ibm.com/blogs/research/wp-content/uploads/2019/09/10IBMQsystems-1024x768.jpg)





Let's go back to classic computer.

Imagine the you have memory space looks like **Yorktown** above.

    0xA (50% noise) - A = (1 + 2) 1 operation
    0xB (5% noise) - B = (3 + 4) 1 operation
    0xC (0% noise) - C = (A + B) 3 operation

We will use this again.

    Now lets say the memory is located at the following qubit

    0xA - 1 qubit
    0xB - 4 qubit
    0xC - 2 qubit

Now let's say we could only apply two variable addition (+) when only the two memory is connected with line.

    The memory space looks like this
    A -  C  - 3
    |   | |   |
    0 -     - B

    So we cannot operate C = (A + B) directly

SWAP 1 - swap A and C and apply (A + B)

    A -  C  - 3            C -  A  - 3
    |   | |   |     ->     |   | |   |
    0 -     - B            0 -     - B

SWAP 2 - swap A and 0 and A and C and apply (A + B)

    A -  C  - 3            0 -  C  - 3          0 -  A  - 3
    |   | |   |     ->     |   | |   |    ->    |   | |   |
    0 -     - B            A -     - B          C -     - B


It seems swap method 1 is seems good because it has less move. But the problem is not that simple.

**Swap noise is also different between memorys(qubits)**

    # Swap 1
    A <-> C noise 70%

    # Swap 2
    A <-> 0 noise 10%
    0 <-> C noise 20%

    YES swap 2 is much better!

#### 4. Gate Optimization


Merging the gates together or removing adjacent identical gate is Gate Optimization. Quantum computer can only use Unitary / Hermitian Operator so gate cancellation between same gate is possible.

    Gate 
    XX = I
    YY = I
    HH = I

You can think this two things in classic computer. So far you changed the **add function** to **+** operator and choose the what memory to use for variable. And you also modified your program to make your algorithm works on a real computer. You had to swap values among memory spaces to move the values to the specific memory space that could execute operation. And every thing was done considering noise. 

**Now this is the last step. Reduce the unnecessary operator**

1. Merging gates


    A = 3 <- 1 + 2
    B = 7 <- 3 + 4
    C = 10 <- 1 + 2 + 3 + 4 = A + B

2. Gate Cancellation


    If A is binary and 0000 0001
    A~~ is same as A

    A~ = 1111 0001
    A~~ = 0000 0001

Gate optimization reduces total gate number so it reduce noise


## 2. How does optimized circuit looks like depends on hardware?

    Circuit optimization is based on hardware information. 
    Check out how the optimized circuit differ based on hardware

[Jupyter notebook](https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/Check_how_circuit_optimized_depends_on_quantum_backend.ipynb)
[colab](https://colab.research.google.com/drive/150ZR9zoIf8XBFpLM9IXAr6rhkSOcScpU)




## 3. Check quantum hardware information

    We now know circuit optimization is based on hardware information
    It's time to look in details. And consider what information to use
    for deep learning 

[jupyter notebook](https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/hardware_information.ipynb)
[colab](https://colab.research.google.com/drive/1m-bRNeopa6mToCv01wy3_LyjQTYZ6Nap)

## 4. How to generate random circuit?

    Now we need to prepare dataset for deep learning model
    And the data should be a lot. We will going to look at
    random circuit generator to generate dataset

[jupyter notebook](https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/Random_quantum_circuit_Generator.ipynb)
[colab](https://colab.research.google.com/drive/1AD5sX89YILQTAsL8Go2IOYXSFx_hEIaS)

## 5. Generate dataset for Deep learning

    Using the random circuit generator and hardware information
    generate dataset as much as we want. And also preprocess
    the dataset to generate encoded dataset

[jupyter notebook](https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/Generate_dataset.ipynb)
[colab](https://colab.research.google.com/drive/1n_zwBDY8UyXgpQedz8sHVXAjgYcGwN91)

## Deep learning Model

[Plan of the model architecture](https://docs.google.com/document/d/1reiGk7PIy-EpJceb2c9FLxVnZgROA5BWVULIMSviZF0/edit?usp=sharing)

    Each Model performance is not good yet. Still have to find out
    the way to train better. But this will show you one approach
    to optimize circuit with deep learning

**The model will be trained for 3 parts**

1. LSTM Autoencoder
2. Layout predicting model
3. Optimized circuit generator



#### Need to be improved
**Trained by 30K dataset**
1. Generated circuit is still unstable. Must find the way to train properly to predict the correct circuit

#### Interesting fact

1. When the dataset increased to 10K to 30K the layout prediction increased from 25% to 60%

## 6. Train LSTM Autoencoder to find circuit manifold

Good link to learn autoencoder

    https://blog.keras.io/building-autoencoders-in-keras.html
    https://lilianweng.github.io/lil-log/2018/08/12/from-autoencoder-to-beta-vae.html
    [Below picture if from this link]


<img src="https://lilianweng.github.io/lil-log/assets/images/autoencoder-architecture.png" width=800px/>

    Train lstm autoencoder to find the circuit manifold.
    This manifold is a space that will be representing the
    quantum circuit itself

[jupyter notebook](https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/LSTM_Autoencoder.ipynb)
[colab](https://colab.research.google.com/drive/1jcYs0p-XZCoSlnf2j8nwzYeiRKL-aBpJ)

## 6-1 (Optional) Estimating Statevector from circuit

    Rather then finding latent space z using autoencoder
    we can make a model that estimate statevector based on
    quantum circuit and use it for further extention models.
    At here we ae going to Look at state vector.

    we are not using statevectors for further model training
    but you can enjoy how deep learning learns the relationship
    between quantum circuit and statevector

[jupyter notebook](https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/predicting_statevectors_from_circuits.ipynb)
[colab](https://colab.research.google.com/github/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/predicting_statevectors_from_circuits.ipynb)

    Check more about the experiment results of this statevector predicting model

<img src="https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/img/research_log.png?raw=true" width=800px>

[experiment log](https://docs.google.com/spreadsheets/d/1LPAHmYtP5d9_qKSbEQmxlSk6NTAQU2ytRe82e3c6dVQ/edit)

#### Interesting fact

**Training performance drastically improved as datasize increase from 1K to 8M**

<img src="https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/img/mse.png?raw=true" width=300px/>

    When we ran the experiments, we found it interesting
    that the neural networks were able to predict the 
    statevectors much better when they were trained on more data.
    The RMSE (a way to measure how different the predicted and actual statevectors are)
    for totally random predictions is 600, while for perfect 
    predictions it would be 0. With 1k data, the RMSE is 320. 
    With 5m data, the RMSE is 120 and with 8m data the RMSE is 70.

## 7. Train Layout predicting Model

    After training the LSTM Autoencoder use trained encoder to
    do transfer learning for layout predicting model 

[jupyter notebook](https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/layout_prediction.ipynb)
[colab](https://colab.research.google.com/drive/1EeWubYnIIkIdkq-5XNMZ68WXw9i1SMIc)

## 8. Train Optimized circuit generating model

    Complete model of previous 2 models. Get the circuit as input
    and generate the optimized model

[juypter notebook](https://github.com/IllgamhoDuck/Quantum-Circuit-Optimization-with-Deep-learning/blob/master/Optimized_circuit_generator.ipynb)
[colab](https://colab.research.google.com/drive/1--9oFkHl7efT0fI9s1JkrBE1uwczt3YU)

## 9. How to check it is optimized?

    After you train your model and now you can compare the
    statevector of original circuit without noise and the 
    statevector of optimized circuit with noise in real
    quantum computer



In [0]:
# You can get code by this sample code
# Sample code not for runnning!

# With out noise
original_state_vector = qiskit.execute(your_quantum_circuit,
                                       qiskit.Aer.get_backend('qasm_simulator')).result().get_counts()

# With noise
noisy_state_vector = qiskit.execute(your_quantum_circuit,
                                       qiskit.Aer.get_backend('REAL QUANTUM COMPUTER')).result().get_counts()

You can compare the two state vector using the following code

In [0]:
from qiskit.test.base import dicts_almost_equal

"""
Check to see how similar the counts from the two runs are, where delta the allowed difference between
the counts. Returns an empty string if they are almost equal, otherwise returns an error message which can
then be printed.
"""
your_circuit_equality = dicts_almost_equal(original_state_vector, noisy_state_vector, delta=1e-8)

    If the statevector is well optimized then it will give no error.

You can do this with transpiler optimizer and compare with your equality. If your optimizer gives more better result then it means your optimizer is much better!

## A. Future work

1. Find the way to train model better
2. Apply reinforcement learning to make the model find the optimized circuit automatically
3. There is no